# API to CSV

---

## https://www.alphavantage.co/

## https://www.kaggle.com/datasets/alistairking/public-company-esg-ratings-dataset



In [1]:
# Dependencies and Setup

import pandas as pd
#import numpy as np
import requests
import json


# Import any API key
from api_keys import alpha_api_key


# Study data files
esg_data_path = "csv_files/data.csv"


# Reading the mouse data and the study results
esg_data = pd.read_csv(esg_data_path)


# Displaying the data table for preview
esg_data.head()

,ticker,name,currency,exchange,industry,logo,weburl,environment_grade,environment_level,social_grade,...,governance_grade,governance_level,environment_score,social_score,governance_score,total_score,last_processing_date,total_grade,total_level,cik
0,dis,Walt Disney Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,https://static.finnhub.io/logo/ef50b4a2b263c84...,https://thewaltdisneycompany.com/,A,High,BB,...,BB,Medium,510,316,321,1147,19-04-2022,BBB,High,1744489
1,gm,General Motors Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Automobiles,https://static.finnhub.io/logo/9253db78-80c9-1...,https://www.gm.com/,A,High,BB,...,B,Medium,510,303,255,1068,17-04-2022,BBB,High,1467858
2,gww,WW Grainger Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Trading Companies and Distributors,https://static.finnhub.io/logo/f153dcda-80eb-1...,https://www.grainger.com/,B,Medium,BB,...,B,Medium,255,385,240,880,19-04-2022,BB,Medium,277135
3,mhk,Mohawk Industries Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Consumer products,https://static.finnhub.io/logo/26868a62-80ec-1...,https://mohawkind.com/,A,High,B,...,BB,Medium,570,298,303,1171,18-04-2022,BBB,High,851968
4,lyv,Live Nation Entertainment Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,https://static.finnhub.io/logo/1cd144d2-80ec-1...,https://www.livenationentertainment.com/,BBB,High,BB,...,B,Medium,492,310,250,1052,18-04-2022,BBB,High,1335258


In [2]:
esg_data.count()


ticker                  722
name                    722
currency                722
exchange                722
industry                709
logo                    702
weburl                  706
environment_grade       722
environment_level       722
social_grade            722
social_level            722
governance_grade        722
governance_level        722
environment_score       722
social_score            722
governance_score        722
total_score             722
last_processing_date    722
total_grade             722
total_level             722
cik                     722
dtype: int64

In [3]:
esg_data.nunique()

ticker                  722
name                    722
currency                  5
exchange                  2
industry                 47
logo                    702
weburl                  706
environment_grade         5
environment_level         3
social_grade              6
social_level              4
governance_grade          4
governance_level          3
environment_score       197
social_score            186
governance_score         80
total_score             441
last_processing_date     30
total_grade               4
total_level               2
cik                     722
dtype: int64

In [ ]:
#ticker "cop" example trial of API pull, using TIME_SERIES_MONTHLY

url = f"https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol=cop&interval=60min&month=2019-1&outputsize=full&apikey={api_key}"

stock_info_return = requests.get(url)
data = stock_info_return.json()
data

In [11]:
#5 year percent change

percent_change_data = []

esg_ticker_series = esg_data['ticker']

api_key = alpha_api_key

print("Beginning Data Retrieval     ")
print("-----------------------------")


for stock in esg_ticker_series:
    
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={stock}&interval=60min&month=2019-1&outputsize=full&apikey={api_key}"
    
    try:
        percent_info_return = requests.get(url)
        data = percent_info_return.json()
        five_yr_start = data['Monthly Time Series']['2019-01-31']['4. close']
        five_yr_end = data['Monthly Time Series']['2024-01-31']['4. close']
        perc_change = (((float(five_yr_end))-(float(five_yr_start)))/(float(five_yr_start)))*100
        annual_change = ((((float(five_yr_end))-(float(five_yr_start)))/(float(five_yr_start)))*100)/5

        percent_change_data.append({"ticker": stock,
                                    "5 YR Open": five_yr_start,
                                    "5 YR Close": five_yr_end,
                                    "Percentage Change": perc_change,
                                    "Annualized % Change": annual_change})

        print(f'Data found for {stock}')

    except:
        print(f'skipping {stock}, data not found')
        

print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
Data found for dis
Data found for gm
Data found for gww
Data found for mhk
Data found for lyv
Data found for lvs
Data found for clx
Data found for aacg
Data found for aal
Data found for aame
Data found for aaoi
Data found for aaon
Data found for aapl
skipping aatc, data not found
skipping aaww, data not found
skipping aaci, data not found
Data found for aadi
Data found for abeo
skipping abnb, data not found
Data found for abio
skipping abmd, data not found
skipping abos, data not found
skipping absi, data not found
skipping abtx, data not found
Data found for abus
skipping abvc, data not found
Data found for acad
skipping acac, data not found
skipping accd, data not found
Data found for acet
skipping abcl, data not found
Data found for abcb
skipping acev, data not found
skipping achl, data not found
Data found for achc
Data found for aciw
Data found for acls
Data found for aciu
Data found for achv
Data found for acmr
Data foun

In [6]:
percent_change_df = pd.DataFrame(percent_change_data)

{'Meta Data': {'1. Information': 'Monthly Prices (open, high, low, close) and Volumes',
  '2. Symbol': 'cop',
  '3. Last Refreshed': '2024-04-04',
  '4. Time Zone': 'US/Eastern'},
 'Monthly Time Series': {'2024-04-04': {'1. open': '128.0000',
   '2. high': '132.4300',
   '3. low': '126.7500',
   '4. close': '131.7500',
   '5. volume': '15546958'},
  '2024-03-28': {'1. open': '113.8000',
   '2. high': '128.6400',
   '3. low': '110.8500',
   '4. close': '127.2800',
   '5. volume': '120434107'},
  '2024-02-29': {'1. open': '112.3400',
   '2. high': '115.2900',
   '3. low': '108.8400',
   '4. close': '112.5400',
   '5. volume': '120426852'},
  '2024-01-31': {'1. open': '117.1700',
   '2. high': '121.2400',
   '3. low': '105.7701',
   '4. close': '111.8700',
   '5. volume': '110673689'},
  '2023-12-29': {'1. open': '115.0300',
   '2. high': '119.6400',
   '3. low': '109.7000',
   '4. close': '116.0700',
   '5. volume': '142651731'},
  '2023-11-30': {'1. open': '119.4800',
   '2. high': '123

In [ ]:
Merge and drop

In [12]:
#ticker "cop" example trial of API pull, using TIME_SERIES_MONTHLY

url = f"https://www.alphavantage.co/query?function=EARNINGS&symbol=cop&apikey={api_key}"

stock_info_return = requests.get(url)
data = stock_info_return.json()
data

{'symbol': 'COP',
 'annualEarnings': [{'fiscalDateEnding': '2024-03-31', 'reportedEPS': '0'},
  {'fiscalDateEnding': '2023-12-31', 'reportedEPS': '8.78'},
  {'fiscalDateEnding': '2022-12-31', 'reportedEPS': '13.49'},
  {'fiscalDateEnding': '2021-12-31', 'reportedEPS': '6'},
  {'fiscalDateEnding': '2020-12-31', 'reportedEPS': '-0.97'},
  {'fiscalDateEnding': '2019-12-31', 'reportedEPS': '3.59'},
  {'fiscalDateEnding': '2018-12-31', 'reportedEPS': '4.54'},
  {'fiscalDateEnding': '2017-12-31', 'reportedEPS': '0.73'},
  {'fiscalDateEnding': '2016-12-31', 'reportedEPS': '-2.66'},
  {'fiscalDateEnding': '2015-12-31', 'reportedEPS': '-1.39'},
  {'fiscalDateEnding': '2014-12-31', 'reportedEPS': '5.31'},
  {'fiscalDateEnding': '2013-12-31', 'reportedEPS': '5.7'},
  {'fiscalDateEnding': '2012-12-31', 'reportedEPS': '6.11'},
  {'fiscalDateEnding': '2011-12-31', 'reportedEPS': '8.77'},
  {'fiscalDateEnding': '2010-12-31', 'reportedEPS': '5.96'},
  {'fiscalDateEnding': '2009-12-31', 'reportedEPS': 

In [13]:
#to run for complete csv

eps_data = []

esg_stock_list = esg_data['ticker']
api_key = alpha_api_key

print("Beginning Data Retrieval     ")
print("-----------------------------")

for stock in esg_stock_list:
    
    url = f"https://www.alphavantage.co/query?function=EARNINGS&symbol={stock}&apikey={api_key}"
    
    try:
        eps_info_return = requests.get(url)
        data = eps_info_return.json()
        eps_2023 = data['annualEarnings'][1]['reportedEPS']
        eps_2022 = data['annualEarnings'][2]['reportedEPS']
        eps_2021 = data['annualEarnings'][3]['reportedEPS']
        eps_2020 = data['annualEarnings'][4]['reportedEPS']
        eps_2019 = data['annualEarnings'][5]['reportedEPS']
        avg_eps = ((float(eps_2023)) + (float(eps_2022)) + (float(eps_2021)) + (float(eps_2020)) + (float(eps_2019)))/5



        eps_data.append({"ticker": stock,
                            "eps 2019": eps_2019, 
                            "eps 2020": eps_2020, 
                            "eps 2021": eps_2021, 
                            "eps 2022": eps_2022, 
                            "eps 2023": eps_2023,
                            "AVG Annual EPS": avg_eps
                           })
        print(f'Data found for {stock}')
    except:
        print(f'skipping {stock}, data not found')



print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
Data found for dis
Data found for gm
Data found for gww
Data found for mhk
Data found for lyv
Data found for lvs
Data found for clx
Data found for aacg
Data found for aal
Data found for aame
Data found for aaoi
Data found for aaon
Data found for aapl
skipping aatc, data not found
skipping aaww, data not found
skipping aaci, data not found
Data found for aadi
Data found for abeo
skipping abnb, data not found
Data found for abio
skipping abmd, data not found
Data found for abos
Data found for absi
skipping abtx, data not found
Data found for abus
Data found for abvc
Data found for acad
skipping acac, data not found
skipping accd, data not found
Data found for acet
Data found for abcl
Data found for abcb
skipping acev, data not found
Data found for achl
Data found for achc
Data found for aciw
Data found for acls
Data found for aciu
Data found for achv
Data found for acmr
Data found for acor
Data found for acnb
Data found for acrs

In [ ]:
eps_df = pd.DataFrame(eps_data)
eps_df

In [ ]:
#merging

esg_complete = pd.merge(esg_data, eps_df, how="left", on=["ticker", "ticker"])
esg_complete.head()